在原来的dataloader的基础上加上ctrl的细胞来训练gears【因为gears本来在训练时就需要加入ctrl】

env: scGPT

# import

In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
from glob import glob
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torch_geometric.data import Data
from torch_geometric.data import DataLoader

import pickle
import sys
import requests

from types import MethodType
import importlib
from scperturb import *
import anndata as ad

In [2]:
import sys
sys.path.append('/data1/lichen/code/single_cell_perturbation/code_other/GEARS')
import gears
import importlib
importlib.reload(gears)
importlib.reload(gears.gears)
importlib.reload(gears.inference)
importlib.reload(gears.utils)
importlib.reload(gears.inference)
# importlib.reload(gears.PertData)
# importlib.reload(gears.GEARS)


from gears import PertData, GEARS
from gears.inference import evaluate, compute_metrics, deeper_analysis, \
                  non_dropout_analysis
                  
from sklearn.metrics import r2_score
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

In [3]:
sys.path.append("/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/")

In [4]:
import v1
from v1.utils import *
from v1.dataloader import *

In [5]:
importlib.reload(v1)
importlib.reload(v1.utils)
importlib.reload(v1.dataloader)

<module 'v1.dataloader' from '/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/v1/dataloader.py'>

# 所有gene pert的数据metadata

In [6]:
gene_pert_dict = \
{'AdamsonWeissman2016_GSM2406675_10X001': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 0},
 'AdamsonWeissman2016_GSM2406677_10X005': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 0},
 'AdamsonWeissman2016_GSM2406681_10X010': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 0},
#  'DatlingerBock2017': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 1,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
 'DatlingerBock2021': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
 'DixitRegev2016': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
#  'FrangiehIzar2021_RNA': {'perturbation_type': 'CRISPR-cas9', # 唯一一个multi celltype，先留着
#   'perturbation_state': 1,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
 
#  'LaraAstiasoHuntly2023_exvivo': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
#  'LaraAstiasoHuntly2023_invivo': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
#  'LaraAstiasoHuntly2023_leukemia': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
#  'LiangWang2023': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 1,
#   'multi_sgRNA': 1},
#  'McFarlandTsherniak2020': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 0},
 'NormanWeissman2019_filtered': {'perturbation_type': 'CRISPRa',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 0},
 'PapalexiSatija2021_eccite_RNA': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
 'PapalexiSatija2021_eccite_arrayed_RNA': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
 'ReplogleWeissman2022_K562_essential': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
#  'ReplogleWeissman2022_K562_gwps': {'perturbation_type': 'CRISPRi', # pert太多了，先不测试
#   'perturbation_state': 1,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
 'ReplogleWeissman2022_rpe1': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
#  'SantinhaPlatt2023': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 0},
 'ShifrutMarson2018': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 1},
 'SunshineHein2023': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 1},
 'TianKampmann2019_day7neuron': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 1},
 'TianKampmann2019_iPSC': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 1},
 'TianKampmann2021_CRISPRa': {'perturbation_type': 'CRISPRa',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
 'TianKampmann2021_CRISPRi': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
 'XuCao2023': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1}}

prefix_list = list(gene_pert_dict.keys())

# gears跑所有dataset

In [12]:
add_control = True
for prefix in prefix_list[1:]:
    
    print('='*20, prefix)
    
    fix_seed(2024)

    # - init para
    data_dir = '/nfs/public/lichen/data/single_cell/perturb_data/scPerturb/raw/scPerturb_rna/statistic_20240520'
    pert_cell_filter = 100 # this is used to filter perts, cell number less than this will be filtered
    seed = 2024 # this is the random seed
    split_type = 1 # 1 for unseen perts; 0 for unseen celltypes
    split_ratio = [0.7, 0.2, 0.1] # train:test:val; val is used to choose data, test is for final validation
    var_num = 5000 # selecting hvg number
    num_de_genes = 20 # number of de genes
    bs_train = 32 # batch size of trainloader
    bs_test = 32 # batch size of testloader
    lr = 1e-4

    tmp_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/scPerturb'
    # save_prefix = f'GEARS'
    save_prefix = f'GEARS_v2-prefix_{prefix}-pert_cell_filter_{pert_cell_filter}-\
seed_{seed}-split_type_{split_type}-var_num_{var_num}-num_de_genes_{num_de_genes}-bs_train_{bs_train}-\
bs_test_{bs_test}'
    save_dir = os.path.join(tmp_dir, prefix, save_prefix)
    # - load pert_data
    pert_data = pickle.load(open(os.path.join(save_dir,'pert_data.pkl'), 'rb'))


    # - get go genes; special set for GEARS
    pert_data.get_gene2go()
    pert_data.set_pert_genes()

    # - get dataset for gears
    pert_data.get_Data_gears(num_de_genes = pert_data.num_de_genes,
                            dataset_name = ['train', 'test', 'val'],
                            add_control = add_control)

    # - add necessary elements for gears
    pert_data.modify_gears()

    # - get dataloader
    trainloader, testloader, valloader = pert_data.get_dataloader(mode='all')
    
    if len(testloader)==0:
        continue

    # - init gears model
    gears_model = GEARS(pert_data, device = 'cuda:1', 
                            weight_bias_track = False, 
                            proj_name = 'pertnet', 
                            exp_name = 'pertnet')
    gears_model.model_initialize(hidden_size = 64)

    # - train gears model
    train_metrics_list, train_metrics_pert_list, val_metrics_list, val_metrics_pert_list, out, out_non_dropout = \
        gears_model.train(epochs = 20, lr = lr)

    model_prefix = f'lr_{lr}' 
    tmp_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark_202410/unseen_perturbation/method/GEARS_v1' # GEARS_v1: with control cells
    save_dir = os.path.join(tmp_dir, prefix)
    # - save results
    os.makedirs(os.path.join(save_dir,f'result_{model_prefix}'),exist_ok=True)

    # -- save pkl
    pickle.dump(train_metrics_list, open(os.path.join(save_dir,f'result_{model_prefix}/train_metrics_list.pkl'), 'wb'))
    pickle.dump(train_metrics_pert_list, open(os.path.join(save_dir,f'result_{model_prefix}/train_metrics_pert_list.pkl'), 'wb'))
    pickle.dump(val_metrics_list, open(os.path.join(save_dir,f'result_{model_prefix}/val_metrics_list.pkl'), 'wb'))
    pickle.dump(val_metrics_pert_list, open(os.path.join(save_dir,f'result_{model_prefix}/val_metrics_pert_list.pkl'), 'wb'))
    pickle.dump(out, open(os.path.join(save_dir,f'result_{model_prefix}/out.pkl'), 'wb'))
    pickle.dump(out_non_dropout, open(os.path.join(save_dir,f'result_{model_prefix}/out_non_dropout.pkl'), 'wb'))

    # -- save model
    gears_model.save_model(os.path.join(save_dir,f'result_{model_prefix}'))

    # -- save plot
    merge_plot(train_metrics_list, 'train',os.path.join(save_dir,f'result_{model_prefix}/train.png'))
    merge_plot(val_metrics_list, 'test',os.path.join(save_dir,f'result_{model_prefix}/test.png'))

    # - empty cuda
    torch.cuda.empty_cache()
    # break

==================== AdamsonWeissman2016_GSM2406677_10X005


Found local copy...
Found local copy...
 25%|██▌       | 1/4 [00:00<00:00,  8.71it/s]

IRE1 | lymphoblasts not in pert_names
ATF6; PERK; IRE1 | lymphoblasts not in pert_names


100%|██████████| 4/4 [00:00<00:00,  8.71it/s]


ATF6; IRE1 | lymphoblasts not in pert_names


...

added save list
==================== TianKampmann2021_CRISPRa


Found local copy...
Found local copy...
 47%|████▋     | 9/19 [00:00<00:00, 23.90it/s]

TUSC1 | iPSC-induced neuron not in pert_names


100%|██████████| 10/10 [00:00<00:00, 30.01it/s]


========== get Data_gears finished!
add adata finished
add condition finished
add set2conditions finished
init changed
this is lichen version 2!
start pearson_cor and X_tr.shape is:  (10000, 5054)


# debug

In [10]:
pert_data.dataset_pert_cell_graphs['train'].keys()

dict_keys(['SNAI1 | lymphoblasts', 'ZNF326 | lymphoblasts', 'BHLHE40 | lymphoblasts', 'SPI1 | lymphoblasts', 'control | lymphoblasts'])

In [8]:
pert_data.dataset_pert_cell_graphs['train']['control | lymphoblasts']

[Data(x=[1, 5001], y=[1, 5001], pert_idx=[1], de_idx=[20], pert='ctrl'),
 Data(x=[1, 5001], y=[1, 5001], pert_idx=[1], de_idx=[20], pert='ctrl'),
 Data(x=[1, 5001], y=[1, 5001], pert_idx=[1], de_idx=[20], pert='ctrl'),
 Data(x=[1, 5001], y=[1, 5001], pert_idx=[1], de_idx=[20], pert='ctrl'),
 Data(x=[1, 5001], y=[1, 5001], pert_idx=[1], de_idx=[20], pert='ctrl'),
 Data(x=[1, 5001], y=[1, 5001], pert_idx=[1], de_idx=[20], pert='ctrl'),
 Data(x=[1, 5001], y=[1, 5001], pert_idx=[1], de_idx=[20], pert='ctrl'),
 Data(x=[1, 5001], y=[1, 5001], pert_idx=[1], de_idx=[20], pert='ctrl'),
 Data(x=[1, 5001], y=[1, 5001], pert_idx=[1], de_idx=[20], pert='ctrl'),
 Data(x=[1, 5001], y=[1, 5001], pert_idx=[1], de_idx=[20], pert='ctrl'),
 Data(x=[1, 5001], y=[1, 5001], pert_idx=[1], de_idx=[20], pert='ctrl'),
 Data(x=[1, 5001], y=[1, 5001], pert_idx=[1], de_idx=[20], pert='ctrl'),
 Data(x=[1, 5001], y=[1, 5001], pert_idx=[1], de_idx=[20], pert='ctrl'),
 Data(x=[1, 5001], y=[1, 5001], pert_idx=[1], de_id

In [9]:
for batch in trainloader:
    break

In [10]:
batch

DataBatch(x=[32, 5001], y=[32, 5001], pert_idx=[32], de_idx=[32], pert=[32], batch=[32], ptr=[33])

In [11]:
batch.pert

['BHLHE40+ctrl',
 'ctrl',
 'ctrl',
 'SPI1+ctrl',
 'ZNF326+ctrl',
 'ctrl',
 'SPI1+ctrl',
 'ctrl',
 'SPI1+ctrl',
 'ctrl',
 'ctrl',
 'ctrl',
 'ctrl',
 'ZNF326+ctrl',
 'ctrl',
 'ZNF326+ctrl',
 'ctrl',
 'ctrl',
 'SPI1+ctrl',
 'ctrl',
 'SNAI1+ctrl',
 'SPI1+ctrl',
 'SPI1+ctrl',
 'ctrl',
 'SPI1+ctrl',
 'ZNF326+ctrl',
 'ZNF326+ctrl',
 'ctrl',
 'SNAI1+ctrl',
 'ctrl',
 'SNAI1+ctrl',
 'ZNF326+ctrl']

In [14]:
# - get total cell graphs for each dataset
pert_cell_graphs = {}
for pert in tqdm(tmp_perturbation_list):
    adata_pert = tmp_adata_split[tmp_adata_split.obs['perturbation_group']==pert]
    Xs = adata_pert.X # ctrl value
    ys = self.adata_split[adata_pert.obs['control_barcode']].X # perturb value
    
    # - get the de_idx for pert
    if 'rank_genes_groups' in adata_pert.uns:
        de_genes = adata_pert.uns['rank_genes_groups']
        self.de = True
    else:
        self.de = False
        num_de_genes = 1
        
    if self.de:
        de_idx = np.where(adata_pert.var_names.isin(
        np.array(de_genes[pert][:num_de_genes])))[0]
    else:
        de_idx = [-1] * num_de_genes

 20%|██        | 1/5 [00:00<00:00, 52.09it/s]


KeyError: "Values ['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None

In [15]:
adata_pert

View of AnnData object with n_obs × n_vars = 1769 × 5001
    obs: 'perturbation', 'read count', 'UMI count', 'tissue_type', 'cell_line', 'cancer', 'disease', 'perturbation_type', 'celltype', 'organism', 'ncounts', 'ngenes', 'percent_mito', 'percent_ribo', 'nperts', 'perturbation_new', 'perturbation_type_new', 'nperts_new', 'celltype_new', 'sgRNA_new', 'perturbation_group', 'data_split', 'retain', 'n_genes', 'control_barcode', 'sgRNA_ID', 'pert_sgRNA'
    var: 'ensembl_id', 'ncounts', 'ncells'
    uns: 'rank_genes_groups', 'pvals', 'pvals_adj', 'scores', 'logfoldchanges'

In [16]:
pert_data.dataset_perturbation_list['train']

['SNAI1 | lymphoblasts',
 'ZNF326 | lymphoblasts',
 'BHLHE40 | lymphoblasts',
 'SPI1 | lymphoblasts']

In [8]:
self = pert_data
dataset_name = ['train', 'test', 'val']

In [12]:
self.dataset_perturbation_list = {} # save perturbation_list for train/test
self.dataset_pert_cell_graphs = {} # save the pert_cell_graphs
for dataset in dataset_name:
    # - get the perturbation list
    tmp_obs = self.adata_split[self.adata_split.obs['data_split']==dataset].obs
    # tmp_obs = tmp_obs[tmp_obs['perturbation_new']!='control']
    tmp_perturbation_list = list(tmp_obs['perturbation_group'].unique()) # record the perturbation pair
    tmp_adata_split = self.adata_split[tmp_obs.index]
    
    contrl_perturbation = [i for i in tmp_perturbation_list if 'control' in i][0]
    tmp_perturbation_list = [i for i in tmp_perturbation_list if i!=contrl_perturbation]
    
    
    break

In [16]:
contrl_perturbation
tmp_perturbation_list, contrl_perturbation

(['SNAI1 | lymphoblasts',
  'control | lymphoblasts',
  'ZNF326 | lymphoblasts',
  'BHLHE40 | lymphoblasts',
  'SPI1 | lymphoblasts'],
 ['control | lymphoblasts'])